In [1]:
!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

     |████████████████████████████████| 993kB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [17]:
fileId = drive.CreateFile({'id': '#'}) 
print(fileId['title'])  # dataset.zip
fileId.GetContentFile('temp.zip')  # Save Drive file as a local file

temp.zip


In [0]:
import pandas as pd
import numpy as np
import os
import keras
from pathlib import Path
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [19]:
!unzip temp.zip -d ./

images = []
labels = []
p = Path('temp')
dirs = p.glob('*')
total = 0
for folder in dirs:
    label = str(folder).split('/')[-1]
    cnt = 0
    print(label)
    # read png file 
    for img_path in folder.glob('*.png'):
        img = image.load_img(img_path, target_size=(96, 96))
        img_array = image.img_to_array(img)
        images.append(img_array)
        labels.append(label)
        cnt += 1
        total += 1
    
    # read jpg files
    for img_path in folder.glob('*.jpg'):
        img = image.load_img(img_path, target_size=(96, 96))
        img_array = image.img_to_array(img)
        images.append(img_array)
        labels.append(label)
        cnt += 1
        total += 1
        
print('There are' ,total,"images")

Archive:  temp.zip
replace ./temp/Squirtle/6b98aace39044b37871110198106dc1a.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./temp/Squirtle/6b98aace39044b37871110198106dc1a.jpg  
  inflating: ./temp/Squirtle/00000063.png  
  inflating: ./temp/Squirtle/00000076.png  
  inflating: ./temp/Squirtle/00000062.png  
  inflating: ./temp/Squirtle/00000089.png  
  inflating: ./temp/Squirtle/3c57b9ec18724dcda5ef0a545b5b50ab.jpg  
  inflating: ./temp/Squirtle/00000116.jpg  
  inflating: ./temp/Squirtle/00000102.jpg  
  inflating: ./temp/Squirtle/68be31b6780d45188c74315746da8d98.jpg  
  inflating: ./temp/Squirtle/00000100.png  
  inflating: ./temp/Squirtle/00000114.png  
  inflating: ./temp/Squirtle/278eab3dfdd24d178b31be5985271978.jpg  
  inflating: ./temp/Squirtle/00000128.jpg  
  inflating: ./temp/Squirtle/00000074.jpg  
  inflating: ./temp/Squirtle/17ca3a97a048407ab853df33db099b3f.jpg  
  inflating: ./temp/Squirtle/00000048.jpg  
  inflating: ./temp/Squirtle/71d3e12f365e460e92999a9790

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


Bulbasaur
Lapras
Pikachu
There are 1225 images


In [0]:
data = np.array(images, dtype="float") / 255.0
labels = np.array(labels)

In [21]:
input_shape = data[0].shape

base_model= VGG16(weights='imagenet',include_top=False, input_shape=input_shape)

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(2048,activation='relu')(x) 
x=Dense(2048,activation='relu')(x) 
preds=Dense(5,activation='softmax')(x) 


model=Model(inputs=base_model.input,outputs=preds)

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step


In [22]:
data.shape
from sklearn.utils import shuffle
data, labels = shuffle(data, labels, random_state=42)
labels

array(['Bulbasaur', 'Bulbasaur', 'Lapras', ..., 'Pikachu', 'Lapras',
       'Pikachu'], dtype='<U10')

In [0]:
labels.shape
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [0]:
for layer in model.layers:
    layer.trainable=False

for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [0]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest")

In [0]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2)

In [27]:
EPOCHS = 10
INIT_LR = 1e-3
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS), metrics=["accuracy"])


H = model.fit(x=trainX, y=trainY,validation_data=(testX, testY), epochs=EPOCHS, verbose=1, batch_size=128)

Instructions for updating:
Use tf.cast instead.
Train on 980 samples, validate on 245 samples
Epoch 1/10
980/980 [==============================] - 126s 129ms/step - loss: 1.8679 - acc: 0.3112 - val_loss: 1.2739 - val_acc: 0.5306
Epoch 2/10
980/980 [==============================] - 123s 125ms/step - loss: 1.0167 - acc: 0.6245 - val_loss: 0.8150 - val_acc: 0.7061
Epoch 3/10
980/980 [==============================] - 122s 125ms/step - loss: 0.7224 - acc: 0.7235 - val_loss: 0.6971 - val_acc: 0.7429
Epoch 4/10
980/980 [==============================] - 122s 125ms/step - loss: 0.5338 - acc: 0.8224 - val_loss: 0.5355 - val_acc: 0.8163
Epoch 5/10
980/980 [==============================] - 123s 126ms/step - loss: 0.3938 - acc: 0.8714 - val_loss: 0.5491 - val_acc: 0.7878
Epoch 6/10
980/980 [==============================] - 122s 125ms/step - loss: 0.3487 - acc: 0.8796 - val_loss: 0.4378 - val_acc: 0.8531
Epoch 7/10
980/980 [==============================] - 123s 125ms/step - loss: 0.2914 - acc